<a href="https://colab.research.google.com/github/HaTT2018/Air_Traffic/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Air Traffic Data and Import Basic Pkgs

In [ ]:
!pip install -U --no-cache-dir gdown --pre
!pip install ipdb

In [ ]:
!gdown --id 12UB_JFNcH2-FTNVgcCcGQD5CD0zYcLUP --no-cookies
!gdown --id 16wihT95GszfotQKv-a7xY6JEWk1abdI3 --no-cookies
!mkdir data
!mv 718598182_T_DB1B_MARKET.csv ./data/718598182_T_DB1B_MARKET.csv
!mv 'Smart Location Data Export.csv' './data/Smart Location Data Export.csv'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipdb

# Load Data

In [4]:
sl_data = pd.read_csv('./data/Smart Location Data Export.csv')

In [5]:
sl_data.head(2)

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,Com_VMT_Per_Worker,VMT_per_worker,VMT_tot_min,VMT_tot_max,VMT_tot_avg,GHG_per_worker,Annual_GHG,SLC_score,Shape_Length,Shape_Area
0,1,481130078254,481130078254,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,21.688738,27.496405,11.442995,82.636303,25.659327,24.499297,6369.817221,77.450956,3110.36082,297836.0831
1,2,481130078252,481130078252,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,21.379832,26.465754,11.442995,82.636303,25.659327,23.580987,6131.056669,78.898635,3519.46911,484945.1466


In [128]:
path = './data/Origin_and_Destination_Survey_DB1BMarket_2019_4.csv'
trip_data = pd.read_csv(path)
airport_lst = trip_data['Origin'].drop_duplicates().sort_values()
airport_lst = list(airport_lst)
print('There are %i departue airports'%len(airport_lst))

There are 437 departue airports


In [124]:
trip_data.head(2)

,ItinID,Origin,Dest,Passengers,MktFare,NonStopMiles
0,20194610480,LAX,JAN,1.0,328.5,1642.0
1,20194610480,JAN,LAX,1.0,328.5,1642.0


# Data Processing

In [129]:
OD = pd.DataFrame(index=airport_lst, columns=airport_lst)


In [140]:
for i in range(len(airport_lst)):
    print('Starting', i, '/ 437')
    for j in range(len(airport_lst)):
        trip_count = trip_data.loc[(trip_data['Origin']==airport_lst[i]) \
                      & (trip_data['Dest']==airport_lst[j])]['Passengers'].sum()
        OD.loc[airport_lst[i], airport_lst[j]] = trip_count
        

Starting 0 / 437


KeyboardInterrupt: 